# Drug synergy
- Bliss synergy paper: https://www.frontiersin.org/articles/10.3389/fphar.2017.00158/full


## Experiment design
Hey Abe, I believe I already shared some papers regarding Bliss synergy calculation with you. Synergy is showing whether or not drug A and drug B have greater killing than drug A alone + drug B alone

CTG stands for cell titer glo which is the luciferase assay used to measure killing

To perform the assay you dose in a grid with increased concentrations of drug A on the x axis and increased concentrations of drug B on the y axis

The visual representation you get from the data is a heat map

Which shows excess killing compared to what is expected from drug A alone + drug B alone

One way to plot the data is using the heat map

You can also present the numerical scores in a table format

HL-60 and KG-1 showed positive results, aka there was synergy between decitabine and RG3039

The other cell lines were negative

    hl60_bliss_mat_6x6.txt  kg1_bliss_mat_6x6.txt
    hl60_ctg_mat_6x6.txt    kg1_ctg_mat_6x6.txt
    hl60_ctg_mat_7x7.txt    kg1_ctg_mat_7x7.txt

These are the data used by the notebook to plot the heat maps

Since we only see positive data with HL60 and KG1

You can see if 6x6 or 7x7 looks better

I thought 6x6 was good

I can clean the tables, but it will have to be over Christmas break

In [1]:
ls synergy/*.txt

synergy/hl60_bliss_mat_6x6.txt  synergy/kg1_bliss_mat_6x6.txt
synergy/hl60_ctg_mat_6x6.txt    synergy/kg1_ctg_mat_6x6.txt
synergy/hl60_ctg_mat_7x7.txt    synergy/kg1_ctg_mat_7x7.txt


In [6]:
!ls synergy/ctg/raw_data_glomax

190216_hl60_d5.xls    190218_kg1_d7.xls
190216_kg1_d5.xls     190218_molm14_d7.xls
190216_molm14_d5.xls  190218_oci2_d7.xls
190216_oci2_d5.xls    190218_oci3_d7.xls
190216_oci3_d5.xls    190218_thp1_d7.xls
190216_thp1_d5.xls    190320_molm14_newdoses_n3_biological_reps_d5.xls
190218_hl60_d7.xls    190423_kasumi1_d5.xls


## Heatmap

In [3]:
%reload_ext rpy2.ipython

In [82]:
%%R 
library(tidyverse)
library(ggplot2)
library(RColorBrewer)
library(viridis)
library(reshape2)
library(patchwork)
# library(ggthemes)

# CTG
CTG stands for cell titer glo which is the luciferase assay used to measure killing


In [164]:
%%R
plot_CTG <- function(ctg_mat,title){
    p <- ggplot(ctg_mat, aes(x = variable, y = dec_dose, fill = value, label = sprintf('%.1f', value))) +
    geom_tile(color = 'white', size = 0.5) +
    theme_minimal() + 
    geom_text(aes(color = threshold), size = 5) +
    theme(plot.title = element_text(face = "bold",size = rel(1.2), hjust = 0.5),
          axis.text.x = element_text(size=10,color = 'black',angle = 45, hjust = 1),
          axis.text.y = element_text(size=10,color = 'black',angle = 45, hjust = 1),
          panel.grid.major = element_blank(),
          text = element_text(size=15),
          legend.title = element_blank(),
          plot.margin=unit(c(10,5,5,5),"mm"),
          # legend.background = element_rect(fill="transparent"),
          # legend.position = element_blank() #'right'
         ) +
    ggtitle(title) + 
    scale_fill_viridis(limits = c(-10,100)) +
    scale_color_manual(values = c('darkblue', 'white')) + 
    xlab(expression('[RG3039] (' * mu * 'M)')) +
    ylab(expression('[Decitabine] (' * mu * 'M)')) +
    guides(color = FALSE)
    
    return(p)
}

In [165]:
%%R 
ctg_mat_melt <- read.table('synergy/ctg_mat_6x6.txt', sep = '\t', header = TRUE, check.names = FALSE)

ctg_mat_melt <- melt(ctg_mat_melt, id.vars = c('line','dec_dose'))
ctg_mat_melt$dec_dose <- as.character(sprintf('%.3f', ctg_mat_melt$dec_dose))
ctg_mat_melt$variable <- as.factor(ctg_mat_melt$variable)
ctg_mat_melt$value = ctg_mat_melt$value * 100

ctg_mat_melt$threshold <- ifelse(ctg_mat_melt$value < 40, 'under', 'over')

In [166]:
%%R 
ctg_mat_melt$line %>% levels

[1] "HL-60"    "Kasumi-1" "KG-1"     "MOLM-14"  "OCI-AML2" "OCI-AML3" "THP-1"   


### ...

In [167]:
%%R 
plot_CTG(ctg_mat_melt %>% filter(line == 'HL-60'),'HL-60') +
plot_CTG(ctg_mat_melt %>% filter(line == 'KG-1'),'KG-1') +
plot_CTG(ctg_mat_melt %>% filter(line == 'MOLM-14'),'MOLM-14') +
plot_CTG(ctg_mat_melt %>% filter(line == 'OCI-AML2'),'OCI-AML2') +
plot_CTG(ctg_mat_melt %>% filter(line == 'OCI-AML3'),'OCI-AML3') +
plot_CTG(ctg_mat_melt %>% filter(line == 'THP-1'),'THP-1') +
plot_CTG(ctg_mat_melt %>% filter(line == 'Kasumi-1'),'Kasumi-1')
ggsave('synergy/CTG.pdf',width = 18, height = 16)

# bliss

In [177]:
%%R 
plot_bliss <- function(bliss_mat,title){
    p <- ggplot(bliss_mat, aes(x = variable, y = dec_dose, fill = value, label = sprintf('%.1f', value))) +
    geom_tile(color = 'white', size = 0.5) +
    theme_minimal() +
    theme(plot.title = element_text(face = "bold",
                                             size = rel(1.2), hjust = 0.5),
          axis.text.x = element_text(size=10,color = 'black',angle = 45, hjust = 0.85),
          axis.text.y = element_text(size=10,color = 'black',angle = 45),# hjust = 0.1),
          panel.grid.major = element_blank(),
          text = element_text(size=15),
          legend.title = element_blank(),
          plot.margin=unit(c(10,5,5,5),"mm"),
          legend.position = 'right'
         ) +
    scale_fill_gradient2(low = '#3182bd', mid = 'gray95', high = '#de2d26', midpoint = 0) +
    geom_text(size = 5) +
    xlab(expression('[RG3039] (' * mu * 'M)')) +
    ylab(expression('[Decitabine] (' * mu * 'M)')) +
    ggtitle(title) + 
    scale_color_manual(values = c('black', 'white'))
    return(p)
}

In [178]:
%%R 
bliss_mat_melt <- read.table('synergy/bliss_mat_6x6.txt', sep = '\t', header = TRUE, check.names = FALSE)
bliss_mat_melt

bliss_mat_melt <- melt(bliss_mat_melt, id.vars = c('line','dec_dose'))
bliss_mat_melt$dec_dose <- as.character(sprintf('%.3f', bliss_mat_melt$dec_dose))
bliss_mat_melt$variable <- as.factor(bliss_mat_melt$variable)

bliss_mat_melt$value = bliss_mat_melt$value * 100
# ctg_mat_melt$threshold <- ifelse(ctg_mat_melt$value < 0.4, 'under', 'over')

In [179]:
%%R 
plot_bliss(bliss_mat_melt %>% filter(line == 'HL-60'),'HL-60') +
plot_bliss(bliss_mat_melt %>% filter(line == 'KG-1'),'KG-1') +
plot_bliss(bliss_mat_melt %>% filter(line == 'MOLM-14'),'MOLM-14') +
plot_bliss(bliss_mat_melt %>% filter(line == 'OCI-AML2'),'OCI-AML2') +
plot_bliss(bliss_mat_melt %>% filter(line == 'OCI-AML3'),'OCI-AML3') +
plot_bliss(bliss_mat_melt %>% filter(line == 'THP-1'),'THP-1') +
plot_bliss(bliss_mat_melt %>% filter(line == 'Kasumi-1'),'Kasumi-1')

ggsave('synergy/bliss.pdf',width = 18, height = 16)

### hl60

In [102]:
# %%R 
# #import dose matrix
# hl60_bliss_mat <- read.table('synergy/hl60_bliss_mat_6x6.txt', sep = '\t', header = TRUE, check.names = FALSE)

# #rename first column
# names(hl60_bliss_mat)[1] <- 'dec_dose'

# #melt data using first column as id variable
# hl60_bliss_mat_melt <- melt(hl60_bliss_mat, id.vars = 'dec_dose')

# #convert decitabine doses (w three decimal places) to characters
# hl60_bliss_mat_melt$dec_dose <- as.character(sprintf('%.3f', hl60_bliss_mat_melt$dec_dose))

# #convert rg3039 doses to factor
# hl60_bliss_mat_melt$variable <- as.factor(hl60_bliss_mat_melt$variable)

# #create variable for value font color
# #hl60_ctg_mat_melt$threshold <- ifelse(hl60_ctg_mat_melt$value < 0.4, 'under', 'over')

# head(hl60_bliss_mat_melt, n = 15)

In [103]:
# %%R 
# p <- ggplot(hl60_bliss_mat_melt, aes(x = variable, y = dec_dose, fill = value, label = sprintf('%.2f', value))) +
# geom_tile(color = 'white', size = 0.5) +
# theme_minimal() +
# theme(plot.title = element_text(face = "bold",
#                                          size = rel(1.2), hjust = 0.5),
#       axis.text.x = element_text(size=10,color = 'black',angle = 45, hjust = 0.85),
#       axis.text.y = element_text(size=10,color = 'black',angle = 45),# hjust = 0.1),
#       panel.grid.major = element_blank(),
#       text = element_text(size=15),
#       legend.title = element_blank(),
#       plot.margin=unit(c(10,5,5,5),"mm"),
#       legend.position = 'right'
#      ) +
# scale_fill_gradient2(low = '#3182bd', mid = 'gray95', high = '#de2d26', midpoint = 0) +
# geom_text(size = 5) +
# xlab(expression('[RG3039] (' * mu * 'M)')) +
# ylab(expression('[Decitabine] (' * mu * 'M)')) +
# ggtitle('hl60 bliss') + 
# scale_color_manual(values = c('black', 'white'))

# ggsave('synergy/hl60_bliss_v4.pdf',width = 6, height = 5.5)

# p

In [105]:
# p <- ggplot(kg1_bliss_mat_melt, aes(x = variable, y = dec_dose, fill = value, label = sprintf('%.2f', value))) +
# geom_tile(color = NA) +
# theme_minimal() +
# xlab(expression('RG3039 (uM)')) +
# ylab(expression('Decitabine (uM)')) +
# theme(axis.text.x = element_text(size = rel(2), color = 'black', angle = 45, hjust = 1),
#       axis.text.y = element_text(size = rel(2), color = 'black'),
#       axis.title.x = element_text(size = rel(1.75), color = 'black'),
#       axis.title.y = element_text(size = rel(1.75), color = 'black'),
#       panel.grid.major = element_blank(),
#       legend.title = element_blank(),
#       legend.position = 'right',
#       legend.text = element_text(size = rel(1.5))) +
# scale_fill_gradient2(low = '#3182bd', mid = 'gray95', high = '#de2d26', midpoint = 0,
#                      guide = guide_colorbar(ticks.linewidth = 2, barheight = 10)) +
# scale_color_manual(values = c('black', 'white'))

# ggsave('kg1_bliss_v2.eps', width = 5.35, height = 4.4)

# p

# 

In [106]:
%%R 
sessionInfo()

R version 3.6.0 (2019-04-26)
Platform: x86_64-redhat-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /data_gilbert/home/aarab/anaconda3/envs/down/lib/libmkl_rt.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] tools     stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
[1] viridis_0.5.1      viridisLite_0.4.0  reshape2_1.4.3     RColorBrewer_1.1-2
[5] ggplot2_3.3.3     

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.6       rstudioapi_0.13  magrittr_2.0.1   tidyselect_0.2.5
 [5] munsell_0.5.0    colorspace_2.0-1 R6_2.5.0         rlang_0.4.11

In [107]:
!date

Tue Dec 14 13:07:42 PST 2021
